In [2]:
import torch
from datasets import load_mnist, get_observation_pixels
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

BATCH_SIZE = 128
train_loader, test_loader, val_loader = load_mnist(BATCH_SIZE)

from config import load_config, get_model_name
config = load_config("linear_vae")

model_name = get_model_name(config)

In [6]:
from trainers import BaseModule
from models.lin_vae import LinearVAE

class LinearVAETrainer(BaseModule):
    def __init__(self, hidden_dims = [512, 256], latent_dim = 2):
        model = LinearVAE(hidden_dims, latent_dim)
        super(LinearVAETrainer, self).__init__(model)
        self.save_hyperparameters()
        
    def forward(self, x, x_cond, y):
        return self.model(x)
    
    def step(self, batch, batch_idx, mode = 'train'):
        x, x_cond, y = batch
        x_hat, mu, log_var, z = self(x, x_cond, y)
        loss = self.model.loss(x, x_hat, mu, log_var)
        self.log_dict({f"{mode}_{key}": val.item() for key, val in loss.items()}, sync_dist=True)
        return loss['loss']
    
    def load_model_checkpoint(model_name):
        path = ('checkpoints/' + model_name + '.ckpt')
        return LinearVAETrainer.load_from_checkpoint(path)


In [ ]:
model = LinearVAETrainer(**config['model_params'])

from trainers import SuperTrainer
trainer = SuperTrainer(**config['trainer_params'], model_name=model_name)
trainer.fit(model, train_loader, val_loader)
#save
trainer.save_model_checkpoint()

In [7]:
model = LinearVAETrainer.load_model_checkpoint(model_name)
model.eval()

from plotting import plot_samples_with_reconstruction
from plotting import plot_latent_images
# plot some samples and their reconstructions

plot_samples_with_reconstruction(model, next(iter(test_loader)))
plot_latent_images(model)

FileNotFoundError: [Errno 2] No such file or directory: '/Home/siv34/edzak2974/projects/MastersThesis/src/checkpoints/linear_vae([512, 256]_2)?max_epochs=50&dataset=MNIST.ckpt'